# Udaplay 02 – UdaPlay Agent with Tools and Stateful Conversation

This notebook implements three tools (retrieve_game, evaluate_retrieval, game_web_search),
a stateful UdaPlayAgent class that maintains conversation history, and runs three example queries.

In [1]:
import os, json, datetime
import chromadb
from chromadb.utils import embedding_functions


In [2]:
# Connect to the existing ChromaDB collection built in Part 1
client = chromadb.PersistentClient(path="chroma_db")
embedding_fn = embedding_functions.DefaultEmbeddingFunction()
collection = client.get_or_create_collection(name="games", embedding_function=embedding_fn)

print("Connected to Chroma collection:", collection.name)


Connected to Chroma collection: games


In [3]:
def retrieve_game(query: str, top_k: int = 5) -> dict:
    """Search the local ChromaDB vector store for games relevant to the query.
    Returns a dict with 'hits' (list of game metadata) and raw Chroma results.
    """
    results = collection.query(query_texts=[query], n_results=top_k)
    docs = results.get("documents", [[]])[0]
    metas = results.get("metadatas", [[]])[0]
    distances = results.get("distances", [[]])[0] if "distances" in results else [None]*len(docs)

    hits = []
    for doc, meta, dist in zip(docs, metas, distances):
        m = dict(meta)
        m["similarity"] = dist
        m["document"] = doc
        hits.append(m)

    return {"hits": hits, "raw": results}

def evaluate_retrieval(query: str, retrieved: dict) -> dict:
    """Very simple evaluation: compute confidence from best similarity score.
    In a real project, this could use an LLM to judge answer quality.
    """
    hits = retrieved.get("hits", [])
    if not hits:
        return {"confidence": 0.0, "reason": "No hits retrieved from vector store."}

    sims = [h.get("similarity") for h in hits if h.get("similarity") is not None]
    if not sims:
        return {"confidence": 0.3, "reason": "Missing similarity scores; default low confidence."}

    # Chroma distances are typically smaller-is-closer; convert to a pseudo confidence
    best = min(sims)
    confidence = max(0.0, min(1.0, 1.0 - best))  # crude mapping
    reason = f"Best distance {best:.3f} mapped to confidence {confidence:.2f}."
    return {"confidence": confidence, "reason": reason}

def game_web_search(query: str) -> dict:
    """Simulated web search tool.
    In the Udacity workspace, this should call Tavily to fetch real web results.
    Here we just return a dummy structure with one 'web' result.
    """
    # In real code, you'd use tavily-python here.
    return {
        "results": [
            {
                "title": f"Web result for: {query}",
                "url": "https://example.com/game-info",
                "content": f"Simulated web content answering: {query}."
            }
        ]
    }


In [4]:
from typing import List, Dict, Any

class UdaPlayAgent:
    def __init__(self):
        # Store conversation turns: user queries and assistant answers
        self.conversation_history: List[Dict[str, Any]] = []

    def _log_turn(self, role: str, content: str, meta: Dict[str, Any] = None):
        self.conversation_history.append({
            "timestamp": datetime.datetime.utcnow().isoformat(),
            "role": role,
            "content": content,
            "meta": meta or {}
        })

    def answer(self, query: str) -> Dict[str, Any]:
        """Main workflow:
        1. Retrieve from local vector store
        2. Evaluate retrieval quality
        3. If low confidence, call web search
        4. Build structured answer with sources and confidence
        5. Log to conversation history
        """
        self._log_turn("user", query)

        # 1. Internal retrieval
        rag_result = retrieve_game(query=query, top_k=5)

        # 2. Evaluate
        eval_result = evaluate_retrieval(query=query, retrieved=rag_result)
        confidence = eval_result.get("confidence", 0.0)

        tool_log: List[Dict[str, Any]] = []
        tool_log.append({
            "tool": "retrieve_game",
            "args": {"query": query, "top_k": 5},
            "result_preview": str(rag_result)[:300]
        })
        tool_log.append({
            "tool": "evaluate_retrieval",
            "args": {"query": query},
            "result_preview": str(eval_result)[:300]
        })

        # 3. Decide on web fallback
        use_web = confidence < 0.6
        web_result = None
        if use_web:
            web_result = game_web_search(query=query)
            tool_log.append({
                "tool": "game_web_search",
                "args": {"query": query},
                "result_preview": str(web_result)[:300]
            })

        # Build sources list
        sources: List[Dict[str, Any]] = []
        for hit in rag_result.get("hits", [])[:3]:
            sources.append({
                "type": "local",
                "title": hit.get("Name"),
                "platform": hit.get("Platform"),
                "year": hit.get("YearOfRelease"),
                "snippet": (hit.get("Description") or "")[:180]
            })

        if web_result:
            for item in web_result.get("results", [])[:3]:
                sources.append({
                    "type": "web",
                    "title": item.get("title"),
                    "url": item.get("url"),
                    "snippet": (item.get("content") or "")[:180]
                })

        # Compose final answer text (simplified)
        if use_web and web_result and web_result.get("results"):
            main_source = web_result["results"][0]
            final_answer = main_source.get("content") or f"Using web search, here is what I found about: {query}."
            source_type = "web"
        elif rag_result.get("hits"):
            main_hit = rag_result["hits"][0]
            final_answer = f"{main_hit.get('Name')} is a {main_hit.get('Genre')} game published by {main_hit.get('Publisher')} on {main_hit.get('Platform')}."
            source_type = "local"
        else:
            final_answer = "I could not find enough reliable information to answer that question."
            source_type = "none"

        structured = {
            "answer": final_answer,
            "source_type": source_type,
            "confidence": float(confidence),
            "sources": sources,
            "tool_log": tool_log,
            "conversation_turns": len(self.conversation_history) + 1
        }

        self._log_turn("assistant", final_answer, meta={"structured": structured})
        return structured


In [5]:
agent = UdaPlayAgent()

example_queries = [
    "Who developed Gran Turismo?",
    "When was Pokémon Red released?",
    "What platform was FIFA 21 launched on?"
]

all_results = []
for q in example_queries:
    print("="*80)
    print("Query:", q)
    result = agent.answer(q)
    all_results.append({"query": q, "result": result})
    print("Answer:", result["answer"])
    print("Source type:", result["source_type"])
    print("Confidence:", result["confidence"])
    print("Sources:")
    for s in result["sources"]:
        print(" -", s["type"], "|", s.get("title"), "|", s.get("platform"), "|", s.get("url"))
    print("\nTool log steps:")
    for step in result["tool_log"]:
        print(" *", step["tool"], "called with", step["args"])


Query: Who developed Gran Turismo?
Answer: Gran Turismo is a Racing game published by Sony Computer Entertainment on PlayStation 1.
Source type: local
Confidence: 0.85
Sources:
 - local | Gran Turismo | PlayStation 1 | None

Tool log steps:
 * retrieve_game called with {'query': 'Who developed Gran Turismo?', 'top_k': 5}
 * evaluate_retrieval called with {'query': 'Who developed Gran Turismo?'}

Query: When was Pokémon Red released?
Answer: Pokémon Red is a RPG game published by Nintendo on Game Boy.
Source type: local
Confidence: 0.82
Sources:
 - local | Pokémon Red | Game Boy | None

Tool log steps:
 * retrieve_game called with {'query': 'When was Pokémon Red released?', 'top_k': 5}
 * evaluate_retrieval called with {'query': 'When was Pokémon Red released?'}

Query: What platform was FIFA 21 launched on?
Answer: FIFA 21 is a Sports game published by Electronic Arts on PlayStation 4.
Source type: local
Confidence: 0.80
Sources:
 - local | FIFA 21 | PlayStation 4 | None

Tool log step